In [ ]:
# CODE TO INCREASE IMAGE SIZE (WHILE KEEPING RESOLUTION)
# IMPLEMENTATION OF CV2s dnn_superres MODEL

In [ ]:
import os
import json 
import requests
import selenium
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
import base64
import time
import urllib.request

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from PIL import Image

from selenium.webdriver.common.keys import Keys

import matplotlib.pyplot as plt
import cv2
from cv2 import dnn_superres

import re


In [ ]:
# Upscale image by factor 4 while keeping resolution.
# SPECIFIY PATH WHERE EDSR_X4.PB MODEL IS SAVED
# SPECIFY PATH WHERE IMAGES SHOULD BE SAVED

def scale_up(folder, file):
    img = cv2.imread(f"./{folder}/{file}")
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"
    sr.readModel(path)
    sr.setModel("edsr",4)
    result = sr.upsample(img)
    cv2.imwrite(f"./{folder}/upscaled_{file}", result)
    

In [ ]:
# Add padding to Thumbnail to (a) make it big enough if not already and (b) make it square.

def add_margin(pil_img, size, color):
    width, height = pil_img.size
    new_width = size
    new_height = size
    
    if (width > new_width) or (height > new_height):
        if width > height:
            new_width = width
            new_height = width
        if height >= width:
            new_width = height
            new_height = height

    elif width > height:
        pil_img = pil_img.resize((round(width*(300/min([width, 300]))), round(height*(300/min([width, 300])))))
        width, height = pil_img.size
    elif height >= width:
        pil_img = pil_img.resize((round(width*(300/min([height, 300]))), round(height*(300/min([height, 300])))))
        width, height = pil_img.size   

    result = Image.new(pil_img.mode, (new_width, new_height), color)
    left = (new_width - width) // 2
    top = (new_height - height) // 2
    result.paste(pil_img, (left, top))
    return result

In [ ]:
def retrieve_images(finder_folder, company, n, ):
    time.sleep(3)
    SAVE_FOLDER = f"Logos/{finder_folder}/{company}/"
    try:
        os.mkdir(SAVE_FOLDER)
    except:
        pass
    GOOGLE_IMAGES = f"https://www.google.com/search?q={company} company logo&rlz=1C5CHFA_enGB937GB937&sxsrf=ALeKk02twWeFWJCESdRh27hZxC0iZwBS6w:1617724623722&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiu0ans_envAhUkT98KHZ7CDiMQ_AUoAXoECAEQAw&biw=714&bih=732"
    driver.get(GOOGLE_IMAGES)
    counter = 0
    image_elements = driver.find_elements(By.CLASS_NAME, 'rg_i')
    for image in image_elements[:n]: 
        if (image.get_attribute('src') is not None):
            
            my_image = image.get_attribute('src').split('data:image/jpeg;base64,')
            filename = SAVE_FOLDER + 'logo'+str(counter)+'.jpeg'
            if(len(my_image) >1): 
                with open(filename, 'wb') as f: 
                    
                    f.write(base64.b64decode(my_image[1]))
                

            else: 
                urllib.request.urlretrieve(image.get_attribute('src'), SAVE_FOLDER + 'logo'+ str(counter)+'.jpeg')
            counter += 1


def re_save_images(finder_folder, company, n):
    SAVE_FOLDER = f"Logos/{finder_folder}/{company}/"
    for i in range(0, n):

        scale_up(SAVE_FOLDER, 'logo'+ str(i)+'.jpeg')  
        
        filename = SAVE_FOLDER + 'upscaled_logo'+ str(i)+'.jpeg'
        filename_2 = SAVE_FOLDER + 'upscaled_logo_new'+ str(i)+'.jpeg'
        filename_3 = SAVE_FOLDER + 'upscaled_logo_new'+ str(i)+'.png'

        im = Image.open(filename) 

        try:
            im_new = add_margin(im, 600, (255, 255, 255))
            im_new.save(filename_2, quality=100)
        except OSError as error :
            scale_up(finder_folder, name)
            im_new = add_margin(im_new, 600, (255, 255, 255))
            im_new.save(filename_2, quality=100)
            im_new.save(filename_3)
            im1 = Image.open(filename_3)
            im1.save(filename_2)
            os.remove(filename_3)
            
        os.remove(filename)

        

In [ ]:
s = """LIST OF COMPANIES SEPERATED BY \n"""

companies = s.split('\n')
n = 3 # NUMBER OF IMAGES TO RETRIEVE PER COMPANY
finder_folder = 'FOLDER TO SAVE IMAGES'

In [ ]:

DRIVER_PATH = 'chromedriver'
driver = webdriver.Chrome(DRIVER_PATH)

In [ ]:
GOOGLE_IMAGES = f"https://www.google.com/search?q=initate&rlz=1C5CHFA_enGB937GB937&sxsrf=ALeKk02twWeFWJCESdRh27hZxC0iZwBS6w:1617724623722&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiu0ans_envAhUkT98KHZ7CDiMQ_AUoAXoECAEQAw&biw=714&bih=732"
driver.get(GOOGLE_IMAGES)

In [ ]:
for company in companies: 
    retrieve_images(finder_folder, company, n)
    re_save_images(finder_folder, company, n)

In [ ]:
driver.quit()